In [ ]:
#install libraries

!pip install scikit-survival
!pip install xgboost
!pip install pycox
!pip install torchtuples

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: osqp
    Found existing installation: osqp 1.0.4
    Uninstalling osqp-1.0.4:
      Successfully uninstalled osqp-1.0.4
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored
import random
import itertools


pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Practicum Models/survival_episodes_cleaned.csv')
df

,randid,start_date,duration,event_observed,NOAAClimRegion,TempZone,HumidZone,bin_PlantSize_kW,plant_age_months,active_snow_tickets,snow_bin_ticket_minutes,snow_affected_assets,snow_production_level,snow_value_mm,total_daily_snow_mm,low_irradiation,cumulative_snow_mm,hurr_bin_ticket_minutes,hurr_affected_assets,hurr_production_level,HurricanePrep,HurricanePostInspection,hurricane,nearest_hurricane,wind_speed_mean,rain_value_mm,nearest_rain,storm_active_tickets,storm_bin_ticket_minutes,storm_affected_assets,storm_production_level,lightning,storm,duration_minutes_storm,nearest_storm,flood,duration_minutes_flood,nearest_flood,rain,weather_onset
0,C2S1,2018-10-13,1,1,West,T6,H4,large,25.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,2.677792,0.017805,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
1,C2S1,2018-11-22,3,1,West,T6,H4,large,27.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,4.930521,1.136330,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
2,C2S1,2018-11-29,3,1,West,T6,H4,large,27.0,No,NaN,NaN,Unknown,0.0,0.0,1,0.0,NaN,NaN,Unknown,No,No,0.0,354,5.715604,8.015464,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
3,C2S1,2018-11-30,2,1,West,T6,H4,large,27.0,No,NaN,NaN,Unknown,0.0,0.0,1,0.0,NaN,NaN,Unknown,No,No,0.0,354,7.684687,0.239059,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
4,C2S1,2018-12-05,2,1,West,T6,H4,large,27.0,No,NaN,NaN,Unknown,0.0,0.0,1,0.0,NaN,NaN,Unknown,No,No,0.0,354,1.672604,0.396306,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7266,P2S1,2019-03-08,3,1,West,T1,H2,large,2.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,4.936458,0.425360,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
7267,P2S1,2019-03-20,10,1,West,T1,H2,large,2.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,3.394271,1.412394,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
7268,P2S1,2019-03-21,9,1,West,T1,H2,large,2.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,4.167917,3.042049,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
7269,P2S1,2019-03-22,8,1,West,T1,H2,large,2.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,4.028187,0.000410,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1


In [ ]:
X = df.drop(columns=["duration", "event_observed", "start_date", "randid"])
y_time = df["duration"]
y_event = df["event_observed"]

X_encoded = pd.get_dummies(X, drop_first=True)

In [ ]:
X.shape, y_time.shape, y_event.shape, X_encoded.shape

((7271, 36), (7271,), (7271,), (7271, 58))

In [ ]:
X_train, X_test, y_train_time, y_test_time, y_train_event, y_test_event = train_test_split(
    X_encoded, y_time, y_event, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train_time.shape, y_test_time.shape, y_train_event.shape, y_test_event.shape

((5816, 58), (1455, 58), (5816,), (1455,), (5816,), (1455,))

In [ ]:
y_train_struct = Surv.from_arrays(y_train_event.astype(bool), y_train_time)
y_test_struct = Surv.from_arrays(y_test_event.astype(bool), y_test_time)

In [ ]:
param_grid = {
    'n_estimators': [100, 200],
    'max_features': ['sqrt', 0.5],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [5, 10],
    'max_depth': [None, 10]}

all_combinations = list(itertools.product(
    param_grid['n_estimators'],
    param_grid['max_features'],
    param_grid['min_samples_split'],
    param_grid['min_samples_leaf'],
    param_grid['max_depth']))

# Shuffle and sample 10 combinations for faster testing
random.seed(42)
param_samples = random.sample(all_combinations, 10)

In [ ]:
best_score = -np.inf
best_params = None
best_model = None

for idx, (n, f, split, leaf, depth) in enumerate(param_samples):
    print(f"\n[{idx+1}/10] Trying: n_estimators={n}, max_features={f}, min_samples_split={split}, min_samples_leaf={leaf}, max_depth={depth}")

    model = RandomSurvivalForest(
        n_estimators=n,
        max_features=f,
        min_samples_split=split,
        min_samples_leaf=leaf,
        max_depth=depth,
        random_state=42,
        n_jobs=-1
    )

    scores = []
    kf = KFold(n_splits=3, shuffle=True, random_state=42)

    for train_idx, val_idx in kf.split(X_train):
        X_fold_train = X_train.iloc[train_idx]
        X_fold_val = X_train.iloc[val_idx]
        y_fold_train = y_train_struct[train_idx]
        y_fold_val = y_train_struct[val_idx]

        model.fit(X_fold_train, y_fold_train)
        preds = model.predict(X_fold_val)
        score = concordance_index_censored(
            y_fold_val['event'], y_fold_val['time'], preds
        )[0]
        scores.append(score)

    mean_score = np.mean(scores)
    print(f"Mean C-index: {mean_score:.4f}")

    if mean_score > best_score:
        best_score = mean_score
        best_params = (n, f, split, leaf, depth)
        best_model = model



[1/10] Trying: n_estimators=100, max_features=sqrt, min_samples_split=10, min_samples_leaf=10, max_depth=10
Mean C-index: 0.7405

[2/10] Trying: n_estimators=100, max_features=sqrt, min_samples_split=5, min_samples_leaf=5, max_depth=None
Mean C-index: 0.7596

[3/10] Trying: n_estimators=200, max_features=sqrt, min_samples_split=10, min_samples_leaf=10, max_depth=10
Mean C-index: 0.7405

[4/10] Trying: n_estimators=100, max_features=0.5, min_samples_split=5, min_samples_leaf=5, max_depth=None
Mean C-index: 0.7499

[5/10] Trying: n_estimators=200, max_features=0.5, min_samples_split=10, min_samples_leaf=10, max_depth=10
Mean C-index: 0.7450

[6/10] Trying: n_estimators=200, max_features=0.5, min_samples_split=5, min_samples_leaf=10, max_depth=10
Mean C-index: 0.7450

[7/10] Trying: n_estimators=100, max_features=sqrt, min_samples_split=10, min_samples_leaf=5, max_depth=None
Mean C-index: 0.7596

[8/10] Trying: n_estimators=200, max_features=0.5, min_samples_split=10, min_samples_leaf=5,

In [ ]:
print("\nBest Parameters:", best_params)
preds_test = best_model.predict(X_test)
c_index_test = concordance_index_censored(y_test_struct['event'], y_test_struct['time'], preds_test)[0]
print(f"Final Test C-index: {c_index_test:.4f}")


Best Parameters: (100, 'sqrt', 5, 5, None)
Final Test C-index: 0.7633
